In [170]:
!pip3 install torch_optimizer torchmetrics
!nvidia-smi

Thu Sep 22 20:16:42 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.82.01    Driver Version: 470.82.01    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    35W / 250W |   8141MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [171]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_optimizer import Ranger
from torchvision import datasets
from torchvision import transforms,models
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split
from torchmetrics import Accuracy
from torch.optim.lr_scheduler import CyclicLR

torch.manual_seed(43)

In [172]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [173]:

def target_trans(target):
  y = -torch.ones(100)
  y[target] = 1

  return y,target 

In [174]:
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(size=[32,32], padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.CenterCrop(size=[32,32]),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

dataset = datasets.CIFAR100(root='data/', download=True, transform=transform_train,target_transform=target_trans)
test_dataset = datasets.CIFAR100(root='data/', train=False, transform=transform_test,target_transform=target_trans)


val_size = 5000
train_size = len(dataset) - val_size
train_ds, val_ds = random_split(dataset, [train_size, val_size])
batch_size=256

train_loader = DataLoader(train_ds, batch_size, shuffle=True, num_workers=4)
valid_loader = DataLoader(val_ds, batch_size, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size, num_workers=4)

Files already downloaded and verified


In [175]:
class Hinge_Loss(torch.nn.Module):
    
    def __init__(self):
        super(Hinge_Loss,self).__init__()
        
    def forward(self,y_p,y_t):

        temp = 1 - y_p * y_t
        
        zero = torch.zeros(y_p.size()[0],y_p.size()[1])
        zero = zero.cuda()
        clamp = torch.max(temp,zero)
    
        total_loss = torch.sum(clamp)/y_p.size()[0]
        return total_loss

In [176]:
num_classes = 100
num_epochs = 100


In [177]:
class VGGForCiFar100(nn.Module):
  def __init__(self, hid_dim=5000, dropout=0.5, n_classes=100, use_fc=True, freeze=False):
    super().__init__()
    self.vgg = models.vgg16(pretrained=True)
    if not use_fc:
      self.vgg.classifier = nn.Linear(512 * 7 * 7, n_classes)
    else:
      self.vgg.classifier = nn.Sequential(
              nn.Linear(512 * 7 * 7, 5000),
              nn.ReLU(True),
              nn.Dropout(p=dropout),
              #nn.Linear(4096, 4096),
              #nn.ReLU(True),
              #nn.Dropout(p=dropout),
              nn.Linear(5000, n_classes),
          )
      
    if freeze:
      for param in self.vgg.features.parameters():
        param.requires_grad = False
      for param in self.vgg.avg_pool.parameters():
        param.requires_grad = False
      for param in self.vgg.flatten.parameters():
        param.requires_grad = False
    

  def forward(self, x):
    return self.vgg(x)

In [178]:

model = VGGForCiFar100(n_classes=num_classes).to(device)
criterion = Hinge_Loss().to(device)
accuracy = Accuracy(num_classes=num_classes).to(device)
optimizer = Ranger(model.parameters(), lr=1e-3, weight_decay=1e-4) 
scheduler = CyclicLR(optimizer, base_lr=1e-6, max_lr=1e-3, step_size_up=len(train_loader)//2, cycle_momentum=False)


In [179]:
model

VGGForCiFar100(
  (vgg): VGG(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (6): ReLU(inplace=True)
      (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): ReLU(inplace=True)
      (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (13): ReLU(inplace=True)
      (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (15): ReLU(inplace=True)
      (

In [180]:

def get_accuracy(output, target):

  batch_size = target.size(0)
  correct=0

  pred = output.max(dim=1)[1]

  correct = pred==target

  acc = correct.float().sum(0)

  return acc/batch_size


In [181]:
total_train_step = len(train_loader)
#print(total_train_step)
total_val_step=len(valid_loader)
BEST_VAL_METRIC = 0
BEST_MODEL = None

for epoch in range(1, num_epochs+1):

    train_loss=0
    train_acc=0.0
    model.train()

    for i, (images, target) in enumerate(train_loader, 1):

        y_trans = target[0]
        y_true = target[1]

        # Move tensors to the configured device
        images = images.to(device)
        y_true = y_true.to(device)
        y_trans = y_trans.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, y_trans)

        train_loss += loss
        #train_acc += get_accuracy(outputs, y_true)
        train_acc += accuracy(outputs, y_true)
        
        
        # Backward and optimize
        loss.backward()
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

    print(f'Epoch [{epoch}/{num_epochs}] - Loss: {(train_loss/total_train_step):.4f}, Accuracy: {(train_acc/total_train_step):.4f}')

    model.eval() 
    # Validation
    with torch.no_grad():
        val_acc = 0
        val_loss=0
        for i, (images, target) in enumerate(valid_loader, 1):

            y_trans = target[0]
            y_true = target[1]

            # Move tensors to the configured device
            images = images.to(device)
            y_true = y_true.to(device)
            y_trans = y_trans.to(device)

            outputs = model(images)
            val_loss += criterion(outputs, y_trans)
            #val_acc += get_accuracy(outputs, y_true)
            val_acc += accuracy(outputs, y_true)

    if val_acc/total_val_step > BEST_VAL_METRIC:
        BEST_VAL_METRIC = val_acc/total_val_step
        BEST_MODEL = model.state_dict() 

    print(f'Accuracy of the network on the 5000 validation images: {(val_acc/total_val_step):.4f}, loss: {(val_loss/total_val_step):.4f}') 

Epoch [1/100] - Loss: 11.2977, Accuracy: 0.0388
Accuracy of the network on the 5000 validation images: 0.1498, loss: 2.3833
Epoch [2/100] - Loss: 2.3892, Accuracy: 0.0819
Accuracy of the network on the 5000 validation images: 0.2240, loss: 2.2210
Epoch [3/100] - Loss: 2.2374, Accuracy: 0.1305
Accuracy of the network on the 5000 validation images: 0.2656, loss: 2.1239
Epoch [4/100] - Loss: 2.1659, Accuracy: 0.1701
Accuracy of the network on the 5000 validation images: 0.2877, loss: 2.0636
Epoch [5/100] - Loss: 2.0689, Accuracy: 0.2226
Accuracy of the network on the 5000 validation images: 0.3299, loss: 1.9896
Epoch [6/100] - Loss: 2.1244, Accuracy: 0.2219
Accuracy of the network on the 5000 validation images: 0.2550, loss: 2.0180
Epoch [7/100] - Loss: 1.9617, Accuracy: 0.2427
Accuracy of the network on the 5000 validation images: 0.3411, loss: 1.8839
Epoch [8/100] - Loss: 1.8773, Accuracy: 0.2884
Accuracy of the network on the 5000 validation images: 0.3820, loss: 1.8306
Epoch [9/100] -

In [182]:
#Testing
model.load_state_dict(BEST_MODEL)

total_test_step=len(test_loader)

with torch.no_grad():
    test_acc=0
    test_loss=0

    for i, (images, target) in enumerate(test_loader, 1):
        
        y_trans = target[0]
        y_true = target[1]
        
        images = images.to(device)
        y_true = y_true.to(device)
        y_trans = y_trans.to(device)

        # Forward pass
        outputs = model(images)
        
        # Loss
        test_loss += criterion(outputs,y_trans)
        test_acc += accuracy(outputs, y_true)

    print(f'Accuracy of the network on test images: {(test_acc/total_test_step):.4f}, loss: {(test_loss/total_test_step):.4f}')

Accuracy of the network on test images: 0.5760, loss: 1.4057
